In [1]:
!pip install -U "langgraph>=0.2.0" \
               "langchain>=0.2.11" "langchain-core>=0.2.33" \
               "langchain-community>=0.2.11" "langchain-openai>=0.2.1" \
               "faiss-cpu" "langchain-text-splitters"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.3
    Uninstalling langgraph-1.0.3:
      Successfully uninstalled langgraph-1.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    summary: str

llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

def assistant_node(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

builder = StateGraph(AgentState)
builder.add_node("assistant", assistant_node)
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

graph = builder.compile()

initial_state = {"messages": [SystemMessage(content="Sos un agente instructor de ajedrez para estudiantes muy basicos o principiantes, responde de forma acotada, responde en JSON con los campos title y description"),
                              HumanMessage(content="que es un enroque??")],
                 "summary": None
                 }
result = graph.invoke(initial_state)
print(result["messages"][-1].content)

{
  "title": "Enroque (castling)",
  "description": "El enroque es un movimiento que involucra al rey y a una torre. El rey se desplaza dos casillas hacia la torre y la torre salta para quedar a su lado. Hay dos tipos: enroque corto (flanco de rey) y enroque largo (flanco de dama). Requisitos: ni el rey ni la torre han movido antes; no hay piezas entre ellos; el rey no puede estar en jaque ni pasar por una casilla atacada ni terminar en jaque."
}


In [4]:
from typing import Optional
from typing_extensions import TypedDict, Annotated
import operator

class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    summary: Optional[str]


# Tip: podés inicializar summary en None en el estado inicial

initial_state = {"messages": [SystemMessage(content="Sos un agente instructor de ajedrez para estudiantes muy basicos o principiantes, responde de forma acotada, responde en JSON con los campos title y description"),
                              HumanMessage(content="que es un enroque??")],
                 "summary": llm.invoke(initial_state["messages"] + [HumanMessage(content="hace un breve resumen de esta conversacion")])
                 }


result = graph.invoke(initial_state)
print(result["messages"][-1].content)
print(result["summary"])

{
  "title": "Enroque (castling)",
  "description": "Movimiento especial del ajedrez: el rey y una torre se mueven a la vez. El rey se desplaza dos casillas hacia la torre y la torre se coloca al lado del rey en el otro extremo. Reglas básicas: ni el rey ni la torre han movido antes; no hay piezas entre ellos; el rey no está en jaque y no cruza casillas atacadas. Hay dos variantes: enroque corto (hacia la torre de la esquina cercana) y enroque largo (hacia la torre de la esquina opuesta). Sirve para poner al rey a salvo y activar la torre."
}
content='{\n  "title": "Resumen breve de la conversación",\n  "description": "La conversación trató sobre el enroque en ajedrez. Preguntaste qué es el enroque; es una jugada especial donde el rey se mueve dos casillas hacia una torre y la torre se coloca a su lado. Requisitos: ni el rey ni la torre han movido previamente, las casillas entre deben estar vacías, y el rey no puede pasar ni terminar en jaque. Existen enroque corto (del rey) y enroque 

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# Corpus mínimo (podés cambiarlo por algo de tu dominio)
raw_docs = [
    "La Federación Uruguaya de Ajedrez (FUA) es el organismo rector del ajedrez en Uruguay, afiliado a la FIDE y al Ministerio de Turismo del país. Fundada en 1926, su sede se encuentra en Canelones 982. La federación está presidida actualmente por el Ing. Carlos Milans y tiene como objetivo organizar y regular las actividades ajedrecísticas del país.",
"Información clave Nombre: Federación Uruguaya de Ajedrez (FUA) Afiliaciones: FIDE (Federación Internacional de Ajedrez) Ministerio de Turismo (Uruguay) Presidente: Ing. Carlos Milans Dirección: Canelones 982, Montevideo Año de fundación: 1926",
"Actividades: La FUA regula y organiza la actividad ajedrecística en Uruguay, lo que incluye la realización de campeonatos nacionales como el de 2024, ganado por Andrés Rodríguez. También se encarga de la reglamentación del ajedrez en el país, incluyendo el ajedrez online y las normativas de torneos."
]

docs = [Document(page_content=t) for t in raw_docs]

# Split en chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
chunks = splitter.split_documents(docs)

# Vector store FAISS
emb = OpenAIEmbeddings()
vs = FAISS.from_documents(chunks, embedding=emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

In [6]:
from langchain_core.tools import tool

@tool
def rag_search(question: str) -> str:
    """
    INFO: devuelve indormacion sobre la federacion de ajedrez del uruguay
    """
    docs = retriever.vectorstore.similarity_search(
        question,
        k=retriever.search_kwargs.get("k", None),
    )
    context = "\n\n".join(d.page_content for d in docs)
    if not context:
        return "No se encuentra informacion en nuestra base de conocimiento sobre esto, responder al usuario que pruebe con otra pegunta."   # mensaje en caso de no encontrar nada
    return context

In [7]:
from datetime import datetime
from langchain_core.tools import tool

FAKE_ORDERS = {
    "ABC123": "En preparación",
    "XYZ999": "Entregado",
}

@tool
def get_order_status(order_id: str) -> str:
    """
    Devuelve el estado de un pedido ficticio dado su ID.
    """
    status = FAKE_ORDERS.get(order_id)
    if status is None:
        return f"No encontré el pedido {order_id}."
    return f"Estado actual del pedido {order_id}: {status}"

@tool
def get_utc_time(_: str = "") -> str:
    """
    Devuelve la hora actual en UTC (formato ISO).
    """
    return datetime.utcnow().isoformat()

In [8]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage

# 1) Lista de tools
tools = [rag_search, get_order_status, get_utc_time]  # o tus propias tools

# 2) LLM con tools
llm_with_tools = ChatOpenAI(model="gpt-5-mini", temperature=0).bind_tools(tools)

def assistant_node(state: AgentState) -> AgentState:
    """
    Nodo de reasoning: decide si responder directo o llamar tools.
    """
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# 3) Nodo de tools
tool_node = ToolNode(tools)

In [9]:
def route_from_assistant(state: AgentState) -> str:
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and last.tool_calls:
        return "tools"
    return END

In [10]:
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    route_from_assistant,
    {
        "tools": "tools",
        END: END
    }
)
builder.add_edge("tools", "assistant")

graph = builder.compile()

In [11]:
from langchain_core.messages import HumanMessage

state = {
    "messages": [
        HumanMessage(content="Hola, ¿qué es el ajedrez? hay alguna federacion en uruguay? quien es el presidente de la FUA?")
    ],
    "summary": None
}

result = graph.invoke(state)
print("Respuesta 1:", result["messages"][-1].content)

Respuesta 1: Hola — te respondo en breve:

1) ¿Qué es el ajedrez?
- El ajedrez es un juego/ deporte de mesa y estrategia entre dos jugadores, que se juega en un tablero de 8x8 casillas (blancas y negras). Cada jugador tiene 16 piezas (rey, dama/​reina, dos torres, dos alfiles, dos caballos y ocho peones) con movimientos y reglas específicas. El objetivo es dar “jaque mate” al rey contrario (ponerlo bajo ataque sin posibilidad de escape). Además de ser un juego de ocio, es una disciplina competitiva con torneos, normas de arbitraje y clasificación internacional (FIDE).

2) ¿Hay alguna federación en Uruguay?
- Sí. El organismo rector del ajedrez en Uruguay es la Federación Uruguaya de Ajedrez (FUA). La FUA organiza los campeonatos nacionales, regula torneos, y representa a Uruguay ante la FIDE y otros organismos.

3) ¿Quién es el presidente de la FUA?
- No tengo a mano un nombre verificado y actualizado en este momento. Mi información con fecha de corte (junio de 2024) no me permite afir

In [12]:
from copy import deepcopy

state2 = deepcopy(result)
state2["messages"].append(HumanMessage(content="Usá tu base de conocimiento y decime en que estado se encuentra la orden ABC123."))

result2 = graph.invoke(state2)
print("Respuesta 2:", result2["messages"][-1].content)

Respuesta 2: La orden ABC123 está: En preparación.

Esto normalmente significa que el pedido está siendo procesado/picked y embalado y aún no ha sido enviado. ¿Querés que te consulte el tiempo estimado de envío, el número de seguimiento cuando esté disponible, o que gestione algún cambio/cancelación?


In [13]:
# 🧪 Parte 6: Memoria ligera (summary) como nodo extra (opcional)

from langchain_core.messages import HumanMessage, SystemMessage

def memory_node(state: AgentState) -> AgentState:

    summary_llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

    previous_summary = state.get("summary", "")

    recent_messages = state["messages"][-6:] if len(state["messages"]) > 6 else state["messages"]

    # Construir el prompt para el resumen
    prompt_parts = []

    if previous_summary:
        prompt_parts.append(f"Resumen previo de la conversación:\n{previous_summary}\n")

    prompt_parts.append("Mensajes recientes:")
    for msg in recent_messages:
        role = "Usuario" if isinstance(msg, HumanMessage) else "Asistente"
        content = msg.content if hasattr(msg, 'content') else str(msg)
        # Limitar el contenido para no hacer el prompt demasiado largo
        content_preview = content[:200] + "..." if len(content) > 200 else content
        prompt_parts.append(f"- {role}: {content_preview}")

    prompt_parts.append("\nResumir en 3 bullets (máximo) lo que el usuario y el asistente acordaron o discutieron hasta ahora:")

    prompt_message = "\n".join(prompt_parts)

    # Invocar el LLM para generar el resumen
    summary_response = summary_llm.invoke([HumanMessage(content=prompt_message)])
    nuevo_summary = summary_response.content

    return {"summary": nuevo_summary}


# Reconstruir el grafo con el nodo de memoria
builder = StateGraph(AgentState)
builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)
builder.add_node("memory", memory_node)

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    route_from_assistant,
    {
        "tools": "tools",
        END: END
    }
)

builder.add_edge("tools", "memory")
builder.add_edge("memory", "assistant")

graph_with_memory = builder.compile()

print("=== Prueba del grafo con nodo de memoria ===\n")

state_test = {
    "messages": [
        HumanMessage(content="Hola, ¿quién es el presidente de la FUA?")
    ],
    "summary": None
}

result_test = graph_with_memory.invoke(state_test)
print("Respuesta:", result_test["messages"][-1].content)
print("\n--- Summary generado ---")
print(result_test["summary"])

# Segunda interacción para ver cómo evoluciona el summary
from copy import deepcopy

state_test2 = deepcopy(result_test)
state_test2["messages"].append(HumanMessage(content="¿Y en qué año se fundó la FUA?"))

result_test2 = graph_with_memory.invoke(state_test2)
print("\n\nRespuesta 2:", result_test2["messages"][-1].content)
print("\n--- Summary actualizado ---")
print(result_test2["summary"])

=== Prueba del grafo con nodo de memoria ===

Respuesta: ¿Te refieres a la Federación Uruguaya de Ajedrez (FUA)? 

No tengo información actualizada y la búsqueda automática no devolvió un nombre claro del presidente actual. Si quieres, puedo:

- Intentar buscarlo de nuevo (necesitaré permiso para hacer una búsqueda en la web).  
- Indicarte dónde verificarlo: la página oficial de la FUA, la ficha de Uruguay en la web de la FIDE, o las redes sociales oficiales de la FUA.  
- O si ya tienes una fuente (artículo, web o red social), puedes compartirla y la reviso contigo.

¿Cómo prefieres que proceda?

--- Summary generado ---
- El usuario preguntó quién es el presidente de la FUA.  
- El asistente señaló que la FUA es el organismo rector del ajedrez en Uruguay y mencionó su afiliación a la FIDE.  
- No se proporcionó el nombre del presidente; las respuestas fueron parciales y repetidas.


Respuesta 2: No tengo esa información verificada en mis datos actuales. Puedo buscarla en la web ahor

In [14]:
# Instalación (ejecutar una sola vez)
!pip install gradio langchain langchain-openai langchain-core

import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage

def format_chat_history(messages):
    history = []
    last_user = None

    for msg in messages:
        if isinstance(msg, HumanMessage):
            last_user = msg.content
        elif isinstance(msg, AIMessage):
            history.append((last_user or "Usuario", msg.content))
            last_user = None

    return history

def run_agent(input_text: str, state: dict):
    # Inicializar estado si es None
    if not state:
        state = {"messages": [], "summary": None}

    # Agregar mensaje del usuario
    state["messages"].append(HumanMessage(content=input_text))

    # Invocar el grafo (asume que 'graph' está definido previamente)
    result = graph.invoke(state)

    # Extraer información de tools usados
    last_msg = result["messages"][-1]
    tools_used = "Sin tools"

    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        tool_names = [call.get("name", "unknown") for call in last_msg.tool_calls]
        tools_used = ", ".join(tool_names)

    # Formatear historial para el chat
    chat_history = format_chat_history(result["messages"])

    # Retornar chat, estado actualizado y log de tools
    return chat_history, result, f"**🛠️ Tools usados:** {tools_used}"

def clear_chat():
    """
    Limpia el estado y el chat
    """
    return [], None, "**🛠️ Tools usados:** Sin tools aún"

# Interfaz Gradio
with gr.Blocks(
    title="Agente LangGraph",
    theme=gr.themes.Soft()
) as ui:

    # Header
    gr.Markdown("""
    # 🤖 Agente Conversacional con LangGraph

    Prueba el agente en tiempo real. Observa qué herramientas utiliza en cada respuesta.
    """)

    # Chatbot principal
    chatbot = gr.Chatbot(
        label="💬 Conversación",
        height=450,
        show_copy_button=True
    )

    with gr.Row():
        prompt = gr.Textbox(
            label="Tu mensaje",
            placeholder="Escribí tu pregunta o solicitud aquí...",
            lines=2,
            scale=4
        )
        send_btn = gr.Button("📤 Enviar", variant="primary", scale=1)

    agent_state = gr.State()

    tools_log = gr.Markdown("**🛠️ Tools usados:** Sin tools aún")

    with gr.Row():
        clear_btn = gr.Button("🗑️ Limpiar conversación", variant="secondary")

    # Enviar mensaje con botón
    send_btn.click(
        fn=run_agent,
        inputs=[prompt, agent_state],
        outputs=[chatbot, agent_state, tools_log],
    ).then(
        fn=lambda: "",  # Limpiar el textbox después de enviar
        outputs=[prompt]
    )

    # Enviar mensaje con Enter
    prompt.submit(
        fn=run_agent,
        inputs=[prompt, agent_state],
        outputs=[chatbot, agent_state, tools_log],
    ).then(
        fn=lambda: "",  # Limpiar el textbox después de enviar
        outputs=[prompt]
    )

    # Limpiar chat
    clear_btn.click(
        fn=clear_chat,
        outputs=[chatbot, agent_state, tools_log]
    )


# esto genera automáticamente una URL pública
ui.launch(
    share=True,      # Genera URL pública (https://xxxxx.gradio.live)
    debug=True,      # Muestra logs de errores
    show_error=True  # Muestra errores en la UI
)

/tmp/ipython-input-4199072225.py:52: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-4199072225.py:65: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-4199072225.py:65: DeprecationWarning: The 'show_copy_button' parameter will be removed in Gradio 6.0. You will need to use 'buttons=["copy"]' instead.
  chatbot = gr.Chatbot(
/tmp/ipython-input-4199072225.py:65: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable t

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a566433a9c765b2199.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a566433a9c765b2199.gradio.live
